In [177]:
# pip install pymongo
# !pip install dnspython

In [178]:
# import requests
# from pymongo import MongoClient
# import json

In [179]:
# client = MongoClient('mongodb://localhost:27017/')
# database = client['IndeedJobListing']
# collection = database['Hello']

In [180]:
# job_search_keyword = ['Python+developer']
# api_url = "https://api.scrapingdog.com/indeed"
# api_key = "659b01009794a261d9eaee40"
# pagination_url = "https://in.indeed.com/jobs?q={}&l=&from=searchOnHP&vjk=71d709239fecfc67"
# job_search_url = pagination_url.format(job_search_keyword[0], 0)

# # Set up the parameters
# params = {"api_key": api_key, "url": job_search_url}
# print(params)
# # Make the HTTP GET request
# response = requests.get(api_url, params=params)



In [181]:
# # Check if the request was successful (status code 200)
# if response.status_code == 200:
#     # Parse the JSON content
#     json_response = response.json()
#     # Store the data in MongoDB
#     # collection.insert_one(json_response)
#     if isinstance(json_response, list):
#         for item in json_response:
#             # Extract 'Salary' information
#             salary_info = item.get('Salary', '')
#             item['Salary'] = salary_info.strip() if salary_info else None
#             collection.insert_one(item)
#     else:
#         # Extract 'Salary' information
#         salary_info = json_response.get('Salary', '')
#         json_response['Salary'] = salary_info.strip() if salary_info else None
#         # If json_response is a single document, insert it directly
#         collection.insert_one(json_response)
#         print("Data inserted into MongoDB successfully.")
#     print(json_response)
# else:
#     print(f"Error: {response.status_code}")
#     print(response.text)




In [182]:
# type(json_response)

In [183]:
# def convert(lst):
#    res_dict = {}
#    for i in range(0, len(lst), 2):
#        res_dict[lst[i]] = lst[i + 1]
#    return res_dict
 
 
# # convert(json_response) = convert(json_response)

In [184]:
    # if isinstance(json_response, list):
    #     for item in json_response:
    #         collection.insert_one(item)
    #     print("Data inserted into MongoDB successfully.")
    # else:
    #     # If json_response is a single document, insert it directly
    #     collection.insert_one(json_response)
    #     print("Data inserted into MongoDB successfully.") 

In [98]:
# # Install if you have never used these: unblock the lines below to install if needed
# !pip install webdriver-manager
# !pip3 install lxml
# !pip3 install selenium
# !pip3 install webdriver_manager
# !pip install --upgrade pip
# !pip install -U selenium
# !pip install msedge-selenium-tools


In [209]:
# --------- import necessary modules -------

# For webscraping
from bs4 import BeautifulSoup

# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait

In [210]:
import selenium

# Check version I am running
selenium.__version__

'4.15.2'

In [211]:
# Selenium 4:

from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [212]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [213]:
# Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
option= webdriver.ChromeOptions()

# Going undercover:
option.add_argument("--incognito")


# # Consider this if the application works and you know how it works for speed ups and rendering!

# option.add_argument('--headless=chrome')

In [214]:
    # Define job search keyword for Data Analyst
    job_search_keyword = ['Python+developer']

    # Finding position, radius=35 miles, sort by date and starting page
    # pagination_url = 'https://www.indeed.com/jobs?q={}&l=' 
    pagination_url = 'https://in.indeed.com/jobs?q={}&l=&from=searchOnHP&vjk=71d709239fecfc67'  

    # Print the pagination URL for Data Analyst
    print(pagination_url.format(job_search_keyword[0], 0))


https://in.indeed.com/jobs?q=Python+developer&l=&from=searchOnHP&vjk=71d709239fecfc67


In [215]:
start = time.time()

job = 'Python+developer'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=option)

driver.get(pagination_url.format(job_search_keyword[0], 0))

# Add a sleep to simulate human-like interactions
sleep(randint(2, 6))

# Find the job count element
job_count_element = driver.find_element(By.CLASS_NAME, 'jobsearch-JobCountAndSortPane-jobCount')

# Extract the job count text
job_count_text = job_count_element.text

# Extract the numerical job count
# job_count = int(job_count_text.split(' ')[0])
# Extract the numerical job count, removing commas
job_count = int(job_count_text.split(' ')[0].replace(',', ''))



# Calculate the maximum iterable pages
max_iter_pgs = job_count // 15

# Continue with the rest of your code if needed

driver.quit()  # Closing the browser session

end = time.time()

print(end - start, 'seconds to complete action!')
print('-----------------------')    
print('Max Iterable Pages for this search:', max_iter_pgs)
 


17.563363790512085 seconds to complete action!
-----------------------
Max Iterable Pages for this search: 612


In [216]:
print(job_count)

9194


In [217]:
def try_except(func, default_value):
    try:
        return func()
    except NoSuchElementException:
        return default_value

start = time.time()

job = 'Python+developer'

job_lst = []
salary_list = []

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=option)
sleep(randint(2, 6))

# Set a limit for the number of pages to print debug messages
debug_pages_limit = 5
pages_processed = 0

for i in range(0, max_iter_pgs):
    driver.get(pagination_url.format(job, '', i * 10))

    print(f"Page {i+1} loaded successfully.")

    sleep(randint(2, 4))

    job_page = driver.find_element(By.ID, "mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME, "job_seen_beacon")

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME, "jobTitle")

        print(f"Processing job: {job_title.text}")

        job_lst.append([
            job_title.text,
            job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("href"),
            job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("id"),      
            # jj.find_element(By.CLASS_NAME,"companyName").text,       
            jj.find_element(By.CLASS_NAME,"company_location").text,
            # try_except(lambda: jj.find_element(By.CLASS_NAME, "companyName").text, None),
            job_title.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        ])

        try:
            salary_list.append(jj.find_element(By.CLASS_NAME, "salary-snippet-container").text)
        except NoSuchElementException:
            try:
                salary_list.append(jj.find_element(By.CLASS_NAME, "estimated-salary").text)
            except NoSuchElementException:
                salary_list.append(None)

    # Increment the pages processed counter
    pages_processed += 1

    # Check if we have reached the debug pages limit
    if pages_processed >= debug_pages_limit:
        print(f"Debug messages truncated after processing {debug_pages_limit} pages.")
        break

driver.quit()

end = time.time()

print(end - start, 'seconds to complete Query!')


Page 1 loaded successfully.
Processing job: Data Engineer III
Processing job: Backend Software Developer (PHP/Python)
Processing job: PHP Developer/Laravel Developer
Processing job: AWS/DevOps Engineer
Processing job: Analyst I - IT Solutions
Processing job: Senior Manager - (Data Security/ Data Management)
Processing job: Python Developer
Processing job: Python Developer
Processing job: Python/Django Developer
Processing job: Python Developer , Associate
Processing job: Python Developer
Processing job: Backend Python Developer - WFH
Processing job: Sr. Python Developer
Processing job: Architect II - Software Engineering
Processing job: Python Developer
Page 2 loaded successfully.
Processing job: Data Engineer III
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Processing job: 
Page 3 loaded successfully.
Processing

In [220]:
print(job_lst[0:4])
print(salary_list[0:4])

first_elements = [sublist[0] for sublist in job_lst]
second_elements = [sublist[1] for sublist in job_lst]
fourth_elements = [sublist[3] for sublist in job_lst]

print(first_elements)
print(second_elements)
print(fourth_elements)
print(salary_list)

list_count1 = len(job_lst)
list_count2= len(salary_list)
list_count3 = len(first_elements)
list_count4 = len(second_elements)
list_count5 = len(fourth_elements)

print(list_count1)
print(list_count2)
print(list_count3)
print(list_count4)
print(list_count5)

[['Data Engineer III', 'https://in.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BdkRqoRBllJ3ABBfj5Fwuh4mkM_IbFmVz4qW6oqmasiKwSJy7uXCuCnXnUyLvXm10yTe9BLdEoNGdyosK9BGtOFYwqQ-mLZ89v9g9f4qjujqcAhRsJUlwUYOmf3WinzxT3dn_cYgCKgP_0Jj2ylNcixGmWKl2wzXyhoFEglp9CCMMbP5l32d_6iIsD4JFEGdF3Mzvpf2_ovVcaBc2wPqMelhGaOJocNiaXHQ2Vu4TSPbpr2q_uo4on2CrroJOTcprQUm6S_1b0g3f4pzFtFXNHfJo4PI52UTJSo7vUYdB8Hu3fuN7vWpn74ZBEKJ0NZzvmGqHexhqFyl_w4pr8I8qFTzYGuv8HR09HSVFZw7aKDEhCCyQAaChoMQsDnY3RwSLInUz2DrdOaloSOQb5pQJ-nKcyTFKssKi7LrUCkh859yqFQgPF_v7cVB-jpSv64KDJOFAo95q87RbK3TrK8qziwQdtCMKV1HJzowXTVjcEnwrAYU9pU7wYp6i-YXAdooV8MQ5F3qVkdt11NGZnA6oTJnhe30aMAW0rJVrry5lbZDbPitK7lELYCSELKeRkShjkw-NM3p13dGsU5xWxzVKqv0OEUsDimxI=&xkcb=SoCF6_M3G1R2pFwHhB0KbzkdCdPP&p=0&fvj=1&vjs=3', 'sj_5d8a5ec3c615e23b', 'Indeed\nHyderabad, Telangana', 'https://in.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BdkRqoRBllJ3ABBfj5Fwuh4mkM_IbFmVz4qW6oqmasiKwSJy7uXCuCnXnUyLvXm10yTe9BLdEoNGdyosK9BGtOFYwqQ-mLZ89v9g9f4qjujqcAhRsJUlwUYOmf3WinzxT3dn_cYgCKgP_0Jj2ylNcixGmWK

In [221]:
client = MongoClient('mongodb://localhost:27017/')
database = client['IndeedJobListing']
collection = database['Namaste']

In [222]:
documents = [{"Job Name": first, "Link": second, "Salaries": third, "Company": fourth} for first, second, third, fourth in zip(first_elements, second_elements, salary_list, fourth_elements)]

# Insert the documents into the collection
collection.insert_many(documents)

# Close the MongoDB connection
# client.close()

InsertManyResult([ObjectId('659c5ffdbc60f96322df0d08'), ObjectId('659c5ffdbc60f96322df0d09'), ObjectId('659c5ffdbc60f96322df0d0a'), ObjectId('659c5ffdbc60f96322df0d0b'), ObjectId('659c5ffdbc60f96322df0d0c'), ObjectId('659c5ffdbc60f96322df0d0d'), ObjectId('659c5ffdbc60f96322df0d0e'), ObjectId('659c5ffdbc60f96322df0d0f'), ObjectId('659c5ffdbc60f96322df0d10'), ObjectId('659c5ffdbc60f96322df0d11'), ObjectId('659c5ffdbc60f96322df0d12'), ObjectId('659c5ffdbc60f96322df0d13'), ObjectId('659c5ffdbc60f96322df0d14'), ObjectId('659c5ffdbc60f96322df0d15'), ObjectId('659c5ffdbc60f96322df0d16'), ObjectId('659c5ffdbc60f96322df0d17'), ObjectId('659c5ffdbc60f96322df0d18'), ObjectId('659c5ffdbc60f96322df0d19'), ObjectId('659c5ffdbc60f96322df0d1a'), ObjectId('659c5ffdbc60f96322df0d1b'), ObjectId('659c5ffdbc60f96322df0d1c'), ObjectId('659c5ffdbc60f96322df0d1d'), ObjectId('659c5ffdbc60f96322df0d1e'), ObjectId('659c5ffdbc60f96322df0d1f'), ObjectId('659c5ffdbc60f96322df0d20'), ObjectId('659c5ffdbc60f96322df0d

In [226]:
# pip install djongo

In [224]:
# settings.py

DATABASES = {
    'default': {
        'ENGINE': 'djongo',
        'ENFORCE_SCHEMA': False,
        'NAME': 'YourMongoDBDatabase',  # Replace with your MongoDB database name
    }
}


In [225]:
# models.py

from django.db import models

class Job(models.Model):
    jobTitle = models.CharField(max_length=255)
    jobLink = models.URLField()
    companyName = models.CharField(max_length=255)
    companyLocation = models.CharField(max_length=255)
    jobDescription = models.TextField()
    salary = models.CharField(max_length=255)
    jobMetaData = models.CharField(max_length=255)
    jobPosting = models.CharField(max_length=255)
    date = models.DateTimeField()

    def __str__(self):
        return self.jobTitle


ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [ ]:
# admin.py

from django.contrib import admin
from .models import Job

admin.site.register(Job)


In [ ]:
python manage.py makemigrations
python manage.py migrate


In [ ]:
python manage.py runserver
